In [1]:
"""
Example demonstrating the Robotics Toolbox adapter usage.

This example shows how to use the RTB adapter to:
1. Initialize the environment
2. Load/create robots and objects
3. Perform basic operations like forward kinematics, pose manipulation, etc.
"""

import numpy as np
import spatialmath as sm
import spatialgeometry as sg
from pandaSim.geometry.rtb_adapter import RoboticsToolboxAdapter
from pandaSim.geometry.utils import convert_pose
from pandaSim.geometry.utils import create_virtual_panda
import trimesh
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
import qpsolvers as qp


# auto reload
%load_ext autoreload
%autoreload 2

[I 07/10/25 18:05:35.044 21800] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
panda_path = '../model/franka_description/robots/frankaEmikaPandaVirtual.urdf'
panda = create_virtual_panda(urdf_path=panda_path)

panda

ERobot: panda, 8 joints (RRRRRRRR), 1 gripper, 2 branches, geometry, collision
┌──────┬──────────────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│ link │         link         │ joint │   parent    │              ETS: parent to link               │
├──────┼──────────────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│    0 │ panda_link0          │       │ BASE        │ SE3()                                          │
│    1 │ panda_link1          │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│    2 │ panda_link2          │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│    3 │ panda_link3          │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│    4 │ panda_link4          │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│    5 │ panda_link5          │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(

In [3]:
adapter = RoboticsToolboxAdapter({
    "realtime": True,
    "rate": 100,
    "browser": 'google-chrome' # "notebook" for visualizing in jupyter notebook
})
    

panda_path = '../model/franka_description/robots/frankaEmikaPandaVirtual.urdf'
panda_virtual = create_virtual_panda(urdf_path=panda_path)
    
env = adapter.get_env
# Open the gripper fingers (from sample code)
if hasattr(panda_virtual, 'grippers') and len(panda_virtual.grippers) > 0:
    panda_virtual.grippers[0].q = [0.035, 0.035]
    
box = sg.Box(scale=[0.1, 0.07, 0.03], 
             color='blue', 
             pose=sm.SE3(0.7, 0, 0.015) * sm.SE3.RPY(0, 0, np.pi/3))



# Create a cylinder
cylinder = sg.Cylinder(radius=0.03, length=0.2, 
                       color='green', 
                       pose=sm.SE3(0.3, 0.3, 0.1))

    
axes = sg.Axes(length=0.1, color='red')


env.add(panda_virtual)
env.add(box)
env.add(cylinder)
env.add(axes)

3

In [5]:
# Direct instantiation
from pandaSim.control import QPController

qp_controller = QPController(
    adapter=adapter,
    lambda_q=0.5,      # Joint velocity minimization weight (λ)
    lambda_m=0.1,      # Manipulability maximization weight
    ps=0.05,           # Stopping distance (ρₛ)
    pi=0.3,            # Influence distance (ρᵢ)
    eta=1.0,           # Damper gain (η)
    end_effector_link='panda_finger_virtual',
    solver='quadprog'
)

# Use the controller
qd, arrived = qp_controller.compute_joint_velocities(panda_virtual, box.T)

QP solver failed, using pseudoinverse fallback
